In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
features=pd.read_csv('/content/features.csv')
stores=pd.read_csv('/content/stores.csv')

In [ ]:
try:
  sales=pd.read_csv('/content/train.csv')
except:
  sales=pd.read_csv('/content/train.csv')

In [ ]:
# 4) Merge: (Sales/train has Store, Dept, Date; Features has Store, Date; Stores has Store)
merged = (sales
          .merge(features, on=['Store', 'Date'], how='left')
          .merge(stores,   on='Store',           how='left'))

In [ ]:
print("First 10 rows:\n", merged.head(10))
print("\nLast 10 rows:\n", merged.tail(10))


First 10 rows:
    Store  Dept        Date  Weekly_Sales  IsHoliday_x  Temperature  \
0      1     1  2010-02-05      24924.50        False        42.31   
1      1     1  2010-02-12      46039.49         True        38.51   
2      1     1  2010-02-19      41595.55        False        39.93   
3      1     1  2010-02-26      19403.54        False        46.63   
4      1     1  2010-03-05      21827.90        False        46.50   
5      1     1  2010-03-12      21043.39        False        57.79   
6      1     1  2010-03-19      22136.64        False        54.58   
7      1     1  2010-03-26      26229.21        False        51.45   
8      1     1  2010-04-02      57258.43        False        62.27   
9      1     1  2010-04-09      42960.91        False        65.86   

   Fuel_Price  MarkDown1  MarkDown2  MarkDown3  MarkDown4  MarkDown5  \
0       2.572        NaN        NaN        NaN        NaN        NaN   
1       2.548        NaN        NaN        NaN        NaN        NaN 

In [ ]:
print("\nShape:", merged.shape)
print("\nInfo:")
print(merged.info())
print("\nDescribe (numeric):\n", merged.describe())


Shape: (421570, 17)

Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 421570 entries, 0 to 421569
Data columns (total 17 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Store         421570 non-null  int64  
 1   Dept          421570 non-null  int64  
 2   Date          421570 non-null  object 
 3   Weekly_Sales  421570 non-null  float64
 4   IsHoliday_x   421570 non-null  bool   
 5   Temperature   421570 non-null  float64
 6   Fuel_Price    421570 non-null  float64
 7   MarkDown1     150681 non-null  float64
 8   MarkDown2     111248 non-null  float64
 9   MarkDown3     137091 non-null  float64
 10  MarkDown4     134967 non-null  float64
 11  MarkDown5     151432 non-null  float64
 12  CPI           421570 non-null  float64
 13  Unemployment  421570 non-null  float64
 14  IsHoliday_y   421570 non-null  bool   
 15  Type          421570 non-null  object 
 16  Size          421570 non-null  int64  
dtypes: bool(2), float64(

In [ ]:
#Identify column types
date_cols = [c for c in merged.columns if merged[c].dtype == 'datetime64[ns]']
num_cols  = merged.select_dtypes(include=['number']).columns.tolist()
cat_cols  = [c for c in merged.columns if c not in num_cols + date_cols]
print("\nDate columns:", date_cols)
print("Numeric columns:", num_cols[:20], "  ")
print("Categorical columns:", cat_cols[:20], "  ")


Date columns: []
Numeric columns: ['Store', 'Dept', 'Weekly_Sales', 'Temperature', 'Fuel_Price', 'MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5', 'CPI', 'Unemployment', 'Size']   
Categorical columns: ['Date', 'IsHoliday_x', 'IsHoliday_y', 'Type']   


In [ ]:
#  8) Unique store types & departments
if 'Type' in merged.columns:
    print("\nUnique store types:", merged['Type'].dropna().unique())
if 'Dept' in merged.columns:
    print("Number of departments:", merged['Dept'].nunique())
    print("Sample departments:", merged['Dept'].dropna().unique()[:20])


Unique store types: ['A' 'B' 'C']
Number of departments: 81
Sample departments: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 16 17 18 19 20 21]


In [ ]:
# 9) Columns likely needing cleaning/type conversion (quick checklist)
needs_cleaning = []
if 'IsHoliday_x' in merged.columns or 'IsHoliday' in merged.columns:
    needs_cleaning.append('IsHoliday → boolean')
if 'Date' in merged.columns:
    needs_cleaning.append('Date → datetime (done)')
for md in [f'MarkDown{i}' for i in range(1,6)]:
    if md in merged.columns:
        needs_cleaning.append(f'{md} → numeric fill')
for c in ['Temperature','Fuel_Price','CPI','Unemployment']:
    if c in merged.columns:
        needs_cleaning.append(f'{c} → numeric fill')
print("\nNeeds cleaning:", needs_cleaning)


Needs cleaning: ['IsHoliday → boolean', 'Date → datetime (done)', 'MarkDown1 → numeric fill', 'MarkDown2 → numeric fill', 'MarkDown3 → numeric fill', 'MarkDown4 → numeric fill', 'MarkDown5 → numeric fill', 'Temperature → numeric fill', 'Fuel_Price → numeric fill', 'CPI → numeric fill', 'Unemployment → numeric fill']


In [ ]:

# -------- Task 2: Data Cleaning --------

df = merged.copy()

# 1) Missing values overview
print("\nMissing values before cleaning:\n", df.isnull().sum().sort_values(ascending=False).head(20))



Missing values before cleaning:
 MarkDown2       310322
MarkDown4       286603
MarkDown3       284479
MarkDown1       270889
MarkDown5       270138
Store                0
IsHoliday_x          0
Dept                 0
Date                 0
Temperature          0
Fuel_Price           0
Weekly_Sales         0
CPI                  0
Unemployment         0
IsHoliday_y          0
Type                 0
Size                 0
dtype: int64


In [ ]:
# 2) Fill numeric columns (Temperature, Fuel_Price, CPI, Unemployment) with median
for col in ['Temperature','Fuel_Price','CPI','Unemployment']:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')
        med = df[col].median()
        df[col] = df[col].fillna(med)


In [ ]:

# 3) Fill MarkDown fields with mean
markdown_cols = [c for c in df.columns if c.lower().startswith('markdown')]
for col in markdown_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')
    mean_val = df[col].mean()
    df[col] = df[col].fillna(mean_val)

# 4) Ensure Date is datetime (idempotent)
if 'Date' in df.columns:
    df['Date'] = pd.to_datetime(df['Date'], errors='coerce')


In [ ]:
# 5) Optional: IsHoliday to boolean
for col in ['IsHoliday','IsHoliday_x','IsHoliday_y']:
    if col in df.columns:
        if df[col].dtype != bool:
            df[col] = df[col].astype('bool')

# 6) Remove duplicates (exact row duplicates)
df = df.drop_duplicates().reset_index(drop=True)

print("\nMissing values after cleaning:\n", df.isnull().sum().sort_values(ascending=False).head(20))
print("\nCleaned shape:", df.shape)


Missing values after cleaning:
 Store           0
Dept            0
Date            0
Weekly_Sales    0
IsHoliday_x     0
Temperature     0
Fuel_Price      0
MarkDown1       0
MarkDown2       0
MarkDown3       0
MarkDown4       0
MarkDown5       0
CPI             0
Unemployment    0
IsHoliday_y     0
Type            0
Size            0
dtype: int64

Cleaned shape: (421570, 17)
